In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from huggingface_hub import login


In [3]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", token= '')
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",token='')

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [55]:
def generate_responses(json_data, batch_size=5):
    responses = []
    tokenizer.pad_token = tokenizer.eos_token

    # Prompt format for clarity
    for item in json_data:
        print(item['Response Formats'])

        # answer = ask_question(item['Query'], tokenizer, model)
        # print(f"Question: {question}")
        # print(f"Answer: {answer}")

        # print(item['Query'], "\n\n")
        # print(item['Context'], "\n\n")
        # print(item.get('Additional Instructions', ''), "\n\n")
        queries_with_context = [
            (
                f"Please provide the answer in the following strict format: {item['Response Formats']}\n"
                f"Question: {item['Query']}\n"
                f"Context Data: {item['Context']}\n"
                f"Additional Instructions: {item.get('Additional Instructions', '')}\n"
            )

        ]

    # Batch processing
    for i in range(0, len(queries_with_context), batch_size):
        batch_questions = queries_with_context[i:i + batch_size]

        # Encode batch
        inputs = tokenizer(batch_questions, return_tensors="pt", padding=True, truncation=True, max_length=1000).to(device)

        # Generate model responses
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2
        )

        # Decode and process responses
        for output, item in zip(outputs, json_data[i:i + batch_size]):
            # decoded_output = tokenizer.decode(output, skip_special_tokens=True)
            answer = ask_question(item['Query'], tokenizer, model)

            # # Extract answer after "Answer:" and format as requested
            # match = re.search(r"Answer:\s*(.+)", decoded_output)
            # answer = match.group(1).strip() if match else decoded_output.strip()

            # Append to responses
            responses.append({
                "query": item['Query'],
                # "input": batch_questions[i:i + batch_size],
                "response": answer
            })

    return responses

In [47]:
def ask_question(question, tokenizer, model):
    # Format the prompt
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate a response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the answer (remove the prompt)
    answer = response.split("Answer:")[-1].strip()
    return answer

In [56]:
json_file = "/formula_calculation.json"
df = pd.read_json(json_file)
json_data = df.to_dict(orient='records')

# Generate responses and save
output_data = generate_responses(json_data)

with open('generated_responses.json', 'w') as f:
    json.dump(output_data, f, indent=2)

print("Responses have been saved to 'generated_responses.json'")

For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"
For numeric values: "Answer: {number}" For percentage values: "Answer: \{number\}\%"


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Responses have been saved to 'generated_responses.json'


In [50]:
question = "What is the value of Amazon.com Inc's Net Profit Margin for the Fiscal Year ending in FY 2019? (Response format: Percentage, e.g., '45.5%')"
answer = ask_question(question, tokenizer, model)
print(f"Question: {question}")
print(f"Answer: {answer}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What is the value of Amazon.com Inc's Net Profit Margin for the Fiscal Year ending in FY 2019? (Response format: Percentage, e.g., '45.5%')
Answer: 45.5%
Explanation: Amazon.com Inc's Net Profit Margin for the Fiscal Year ending in FY 2019 is 45.5%.
